# Using data on a local machine

In [1]:
# Import libraries
import os
import glob
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage
import pandas as pd
import datetime as dt

In [2]:
# Set input directory, and change working directory
inDir = 'C:/Users/kayle/Desktop/TA&BA/'  # IMPORTANT: Update to reflect directory on your OS
os.chdir(inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(inDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

# TA dataset

In [3]:
FMFiles = glob.glob('VNP14A1.001_FireMask_****.tif')             # Search for and create a list of EVI files
FMqualityFiles =glob.glob('VNP14A1.001_QA_**.tif')  # Search the directory for the associated quality .tifs
FMlut = glob.glob('VNP14A1-001-QA-lookup.csv')                                        # Search for look up table 
FM_QA_lut = pd.read_csv(FMlut[0])                                    # Read in the lut

# Include good quality based on land/water state
FM_QA_lut = FM_QA_lut[FM_QA_lut['land/water state'].isin(['coast', 'land'])]

FMgoodQuality = list(FM_QA_lut['Value']) # Retrieve list of possible QA values from the quality dataframe

# BurnedArea

In [4]:
BAFiles = glob.glob('MCD64A1.006_Burn_Date_**.tif') # Search for and create a list of BA files
BAqualityFiles =glob.glob('MCD64A1.006_QA_**.tif')    # Search the directory for the associated quality .tifs
BAlut = glob.glob('MCD64A1-006-QA-lookup.csv')                     # Search for look up table 
BA_QA_lut = pd.read_csv(BAlut[0])     # Read in the lut

# Include good quality based on MODLAND
BA_QA_lut = BA_QA_lut[BA_QA_lut['Valid data'].isin([True])]

# Special circumstances unburned
SP =["Too few training observations or insufficient spectral separability between burned and unburned classes"]
BA_QA_lut = BA_QA_lut[~BA_QA_lut['Special circumstances unburned'].isin(SP)]
BA_QA_lut

BAgoodQuality = list(BA_QA_lut['Value']) # Retrieve list of possible QA values from the quality dataframe

In [5]:
def getBAFile(BAF, month, year):
    for x in BAF:
        # File name metadata:
        BAproductId = x.split('_')[0]                                     # First: product name
        BAlayerId = x.split(BAproductId + '_')[1].split('_doy')[0]        # Second: layer name
        BAyeardoy = x.split(BAlayerId+'_doy')[1].split('_aid')[0]         # Third: date
        file_year = dt.datetime.strptime(BAyeardoy, '%Y%j').year
        file_month = dt.datetime.strptime(BAyeardoy, '%Y%j').month
        if file_year==year  and file_month == month:
            return(x) 

# Masking TA by the BurnedArea using only goodQuality data

In [6]:
TA_result = []
#Process through 731 FM files
for i in range(0, len(FMFiles)):
    FM = gdal.Open(FMFiles[i])                    # Read file in, starting with MOD13Q1 version 6
    FMBand = FM.GetRasterBand(1)                  # Read the band (layer)
    FMData = FMBand.ReadAsArray().astype('float') # Import band as an array with type float
    
    FMquality = gdal.Open(FMqualityFiles[i])                       # Open the first quality file
    FMqualityData = FMquality.GetRasterBand(1).ReadAsArray()       # Read in as an array
    FMquality = None 
        
    # File name metadata:
    FMproductId = FMFiles[i].split('_')[0]                                      # First: product name
    FMlayerId = FMFiles[i].split(FMproductId + '_')[1].split('_doy')[0]        # Second: layer name
    FMyeardoy = FMFiles[i].split(FMlayerId+'_doy')[1].split('_aid')[0]         # Third: date
    FM_aid = FMFiles[i].split(FMyeardoy+'_')[1].split('.tif')[0]                # Fourth: unique ROI identifier (aid)
    FM_date = dt.datetime.strptime(FMyeardoy, '%Y%j').strftime('%m/%d/%Y')       # Convert YYYYDDD to MM/DD/YYYY
    FM_year = dt.datetime.strptime(FMyeardoy, '%Y%j').year
    FM_month = dt.datetime.strptime(FMyeardoy, '%Y%j').month   
    
    BAFileName = getBAFile(BAFiles, FM_month, FM_year)
    BAQualityFileName = getBAFile(BAqualityFiles, FM_month, FM_year)
    if not BAFileName in (None, ''):
        BA = gdal.Open(BAFileName)      # Read file in, starting with MCD64A1 version 6
        BABand = BA.GetRasterBand(1)                                 # Read the band (layer)
        BAData = BABand.ReadAsArray().astype('float')                # Import band as an array with type float  
        
        BAquality = gdal.Open(BAQualityFileName)   # Open the first quality file
        BAqualityData = BAquality.GetRasterBand(1).ReadAsArray()                # Read in as an array
        BAquality = None 

        # File Metadata
        BA_meta = BA.GetMetadata()                        # Store metadata in dictionary
        FM_meta = FM.GetMetadata()                      # Store metadata in dictionary

        # Band metadata
        BAFill = BABand.GetNoDataValue()                  # Returns fill value
        BAStats = BABand.GetStatistics(True, True)        # returns min, max, mean, and standard deviation
        BA = None                                         # Close the GeoTIFF file

        # Band metadata
        FMFill = FMBand.GetNoDataValue()                # Returns fill value
        FMStats = FMBand.GetStatistics(True, True)      # returns min, max, mean, and standard deviation
        FM = None                                        # Close the GeoTIFF file

        BAscaleFactor = float(BA_meta['scale_factor'])    # Search the metadata dictionary for the scale factor 
        BAunits = BA_meta['units']                        # Search the metadata dictionary for the units
        BAData[BAData == BAFill] = np.nan                 # Set the fill value equal to NaN for the array
        BAScaled = BAData * BAscaleFactor                 # Apply the scale factor using simple multiplication

        FMscaleFactor = float(FM_meta['scale_factor'])  # Search the metadata dictionary for the scale factor 
        FMunits = FM_meta['units']                      # Search the metadata dictionary for the units
        FMData[FMData == FMFill] = np.nan              # Set the fill value equal to NaN for the array
        FMScaled = FMData * FMscaleFactor              # Apply the scale factor using simple multiplication

        BA_masked = np.ma.MaskedArray(BAScaled, np.in1d(BAqualityData, BAgoodQuality, invert = True))    # Apply QA mask to the BA data
        FM_masked = np.ma.MaskedArray(FMScaled, np.in1d(FMqualityData, FMgoodQuality, invert = True))# Apply QA mask to the EVI data
        BAVal = 0
        
        BA_resampled = scipy.ndimage.zoom(BA_masked,0.5, order=0)   # Resample by a factor of 2 with nearest neighbor interpolation
        new_col = np.sum(BA_resampled,1).reshape((BA_resampled.shape[0],1))
        BA_resampled =  np.append(BA_resampled,new_col,1)
        
        FM_BA = np.ma.MaskedArray(FM_masked, np.in1d(BA_resampled, BAVal, invert = True)) # Mask array, include only BurnArea
    
        TA_result.append([FM_date,FM_BA.max()])
TA_df = pd.DataFrame(TA_result, columns=["Date","TA"])


In [7]:
TA_df.shape

(731, 2)

In [8]:
TA_df

Date   TA
0    01/01/2019  8.0
1    01/02/2019  9.0
2    01/03/2019  9.0
3    01/04/2019  8.0
4    01/05/2019  5.0
..          ...  ...
726  12/27/2020  9.0
727  12/28/2020  9.0
728  12/29/2020  9.0
729  12/30/2020  9.0
730  12/31/2020  9.0

[731 rows x 2 columns]

In [9]:
TA_df.describe()

TA
count  731.000000
mean     8.373461
std      1.239225
min      5.000000
25%      8.000000
50%      9.000000
75%      9.000000
max      9.000000

In [10]:

TA_df.to_csv('TA.csv'.format(outDir), index = False)   # Export TA_df to CSV